# Leonardo F. de Souza

## Visualization with Dash App - Investment Analysis Example

##### October 28th, 2022

###### Leonardo.deSouza@coyotes.usd.edu
###### (605)661-3867
-------------------------------------------------------------------------------------------------------------------------------

### Notes:
1. This visualization uses different managers/securities;
2. The benchmark composition is different and not as accurate as the one shared in the other notebook;
3. This visualization assumes that the new manager composes a different index subcategory;
4. The efficient frontier is calculated outside of a callback look; thus, it does not show the impact of adding an additional manager/security;
5. The purpose of this visualization is to show how the calculations presented previously can be incorporated in a vizualization tool, which might be faster than running the notebook (i.e., use Microsoft Task Scheduler to run the code at a pre-determined time);
6. Feel free to test, change, and use the code as you wish.

#### Modules

In [1]:
#### You might need to pip upgrade poltly and dash
import yfinance as yf
import dash_daq as daq
from jupyter_dash import JupyterDash
import pandas_datareader
from plotly.subplots import make_subplots
from dash.dependencies import Input, Output
from yahoo_fin import stock_info as si
import pandas as pd
import plotly.figure_factory as ff
import plotly.express as px
from plotly.express import *
import plotly.graph_objects as go
import numpy as np
import pandas_datareader.data as web
import datetime as dt
from pytickersymbols import PyTickerSymbols
from yahooquery import Ticker
import pandas_ta as ta
import dash
from dash import dcc, html, dash_table
import plotly
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template
import requests
import json
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
import yfinance as yf
import numpy as np
import plotly.express as px
from plotly.express import *
import plotly.graph_objects as go
import plotly.figure_factory as ff
from dash_bootstrap_templates import load_figure_template
from plotly.subplots import make_subplots
from scipy.optimize import minimize
import getFamaFrenchFactors as gff
from scipy.stats import linregress
import statsmodels.api as sm
import statsmodels.formula.api as smf
from math import e
from scipy import stats
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')
#--------------------------------------
layout = load_figure_template('DARKLY')

#### Data
###### Let's assume that this is our current portfolio, equally weighted. Our Benchmark is the Russell 3000 (IWV ETF as a proxy)

In [2]:
interval = '1mo'
end = dt.datetime.today()
start = end - relativedelta(years=5)
Managers = ['VOX', 'VDC','VDE','VEU', 'VNQI', 'IWV']

# Let our Portfolio Value be $100,000,000:
P = 100000000

Managers = yf.download(Managers, interval=interval, start=start)['Adj Close'] # if available in Excel, use pd.read_excel

df = pd.DataFrame(Managers)
df.dropna(how='all',axis=0, inplace=True)
df.dropna(how='all',axis=1, inplace=True)

# Let w be the weights of an equally weighted portfolio:
w = pd.Series([(1/(len(df.columns)-1))*P for x in range(len(df.columns)-1)]) # funding of each manager
w = [weight/P for weight in w] # weights
print(w)
print(sum(w)) # must be 1

#Returns
df = np.log(df) - np.log(df.shift(1))
df

[*********************100%***********************]  6 of 6 completed
[0.2, 0.2, 0.2, 0.2, 0.2]
1.0


,IWV,VDC,VDE,VEU,VNQI,VOX
Date,,,,,,
2017-10-01,NaN,NaN,NaN,NaN,NaN,NaN
2017-11-01,NaN,0.054245,NaN,NaN,NaN,NaN
2017-12-01,0.007042,0.013165,0.046121,0.011765,0.000165,-0.005471
2018-01-01,0.054671,0.024106,0.037383,0.063994,0.084540,0.023243
2018-02-01,-0.037731,-0.077303,-0.113591,-0.054538,-0.073120,-0.058863
...,...,...,...,...,...,...
2022-06-01,-0.091082,-0.033166,-0.197916,-0.091372,-0.077764,-0.094314
2022-07-01,0.093288,0.033688,0.110210,0.044944,0.036593,0.038391
2022-08-01,-0.038110,-0.015721,0.031507,-0.044945,-0.057594,-0.035274


### Possible future managers
##### Let's assume that we would like to see how replacing or adding a new manager to our portfolio could change our performance. Without manager data, let's use any of the 3000 stocks that compose the Russell 3000 as a "proxy" - this step could easily be replaced by an Excel sheet with the managers returns by using "pd.read_excel()"

In [3]:
url = 'https://www.ishares.com/us/products/239714/ishares-russell-3000-etf/us/products/239714/ishares-russell-3000-etf/1467271812596.ajax?tab=all&fileType=json'
r = requests.get(url)
r.encoding='utf-8-sig'
jsonData = json.loads(r.text)

rows = []
for each in jsonData['aaData']:
    row = {'Ticker':each[0],
      'Name':each[1],
      'Sector':each[2],
#      'Asset Class':each[3],
#      'Market Value':each[4]['display'],
#      'Market Value Raw':each[4]['raw'],
     'Weight (%)':each[5]['display'],
#      'Weight (%) Raw':each[5]['raw'],
#      'Notaional Value':each[6]['display'],
#      'Notaional Value Raw':each[6]['raw'],
#      'Nominal':each[7]['display'],
#      'Nominal Raw':each[7]['raw'],
#      'ISIN':each[8],
#      #'Price':each[9]['display'],
#      #'Price Raw':each[9]['raw'],
#      'Location':each[10],
#      'Exchange':each[11],
      'Market Currency':each[12]
          }
     
    rows.append(row)
     
NM = pd.DataFrame(rows)


## Tickers:
Tickers = NM['Ticker']

ticker_options = []
for Ticker in Tickers.unique():
    ticker_options.append({'label': str(Ticker), 'value': Ticker})
NM
#[:-1:]

,Ticker,Name,Sector,Weight (%),Market Currency
0,AAPL,APPLE INC,Information Technology,6.00,United States
1,MSFT,MICROSOFT CORP,Information Technology,4.54,United States
2,AMZN,AMAZON COM INC,Consumer Discretionary,2.70,United States
3,TSLA,TESLA INC,Consumer Discretionary,1.50,United States
4,GOOGL,ALPHABET INC CLASS A,Communication,1.50,United States
...,...,...,...,...,...
2570,GTXI,GTXI INC - CVR,Health Care,0.00,United States
2571,P5N994,Petrocorp Inc Escrow,Energy,0.00,United States
2572,TFM,FRESH MARKET ESCROW,Consumer Staples,0.00,United States
2573,ESZ2,S&P500 EMINI DEC 22,Cash and/or Derivatives,0.00,-


### Our current efficient frontier should look like this:

In [4]:
dff = df[['VOX', 'VDC','VDE','VEU', 'VNQI']] # This depends on whether we want to include all of these stocks
dff = dff[1::]
# Let's say our returns are:
df_AvgRet = ((np.prod(1+dff))**(12/((dff.count()))))-1

#The Standard deviation of monthly returns:
df_Stdev = ((dff.std())*(np.sqrt(12)))

# The Variance
df_Var = dff.var()*12

# And the Covariance
df_Cov = dff.cov()*12

def minVar(Weights1):
    W_opt1 = np.array(Weights1)
    # Our returns are the sumproduct of average annualized returns and their respective weight
    Returns_opt1 = np.sum(df_AvgRet * W_opt1)
    # And the standard deviation is the sqrt of the matrix multiplication 
    Stdev_opt1 = np.sqrt(np.dot(W_opt1.T, np.dot(df_Cov, W_opt1)))
    return Stdev_opt1

#Making sure our weights sum to 1
def SumIsOne1(Weights1):
    return np.sum(Weights1)-1

# First let's say we have equal weights
w0 = pd.Series([1/len(dff.columns) for x in range(len(dff.columns))])
# And they can either be 0 or 1
bounds1 = pd.Series([(0,1) for x in range(len(dff.columns))]) #-1,2 for long-short

# Our constraint is that the sum of weights should be equal to 1
constraints1 = ({'type':'eq', 'fun': SumIsOne1})
# Now let's minimize our standard deviation, using our "guess" that the weights are equal, and the bound 0,1
Var1 = minimize(minVar, w0, method='SLSQP', bounds=bounds1,
                       constraints=constraints1)

# Our weights should be:
W = Var1['x']
N = dff.columns
Weights = [W, N]
Weights = pd.DataFrame(Weights).T
Weights.columns = ['Weight', 'Security']
print(round(Weights['Weight'].sum(), 3))
Weights

def opt_sd1(W):
    Stdev_opt = np.sqrt(np.dot(W.T, np.dot(df_Cov, W)))
    return Stdev_opt

def opt_ret1(W):
    Ret_opt = np.sum(df_AvgRet * W)
    return Ret_opt

W = Var1['x']

MV_Sd = opt_sd1(W)
MV_Ret = opt_ret1(W)
MV_Port = [MV_Sd, MV_Ret]
MV_Port = pd.DataFrame(MV_Port).T
MV_Port.columns = ['Std. Dev', 'Returns']

MV_Port
rf = 0.0019
def negShar(Weights1, sign=-1.0):
    W_opt1 = np.array(Weights1)
    Returns_opt1 = np.sum(df_AvgRet * W_opt1)
    Stdev_opt1 = np.sqrt(np.dot(W_opt1.T, np.dot(df_Cov, W_opt1)))
    # Now we should maximize our Sharpe. Thus, minimize -Sharpe
    Sharpe_opt1 = (Returns_opt1)/Stdev_opt1
    return sign*Sharpe_opt1

def SumIsOne1(Weights1):
    return np.sum(Weights1)-1

w0 = pd.Series([1/len(dff.columns) for x in range(len(dff.columns))])
bounds1 = pd.Series([(0,1) for x in range(len(dff.columns))]) ### Change to -1,2 for long-short
constraints1 = ({'type':'eq', 'fun': SumIsOne1})
Sharp_W1 = minimize(negShar, w0, method='SLSQP', bounds=bounds1,
                       constraints=constraints1)


W1 = Sharp_W1['x']
N = dff.columns
Weights1 = [W1, N]
Weights1 = pd.DataFrame(Weights1).T
Weights1.columns = ['Weight', 'Security']
print(round(Weights1['Weight'].sum(), 3))
Weights1
def opt_sd(W1):
    Stdev_opt = np.sqrt(np.dot(W1.T, np.dot(df_Cov, W1)))
    return Stdev_opt

def opt_ret(W1):
    Ret_opt = np.sum(df_AvgRet * W1)
    return Ret_opt

W1 = Sharp_W1['x']

S_Sd = opt_sd(W1)
S_Ret = opt_ret(W1)
S_Port = [S_Sd, S_Ret]
S_Port = pd.DataFrame(S_Port).T
S_Port.columns = ['Std. Dev', 'Returns']

S_Port
# Let the max and min returns equal to:
max_ret1 = df_AvgRet.max()
min_ret1 = df_AvgRet.min()

# Returns num evenly spaced samples, calculated over the interval (3,000)
returns1 = np.linspace(min_ret1, max_ret1,3000)

vol_opt1 = []

def minStdev1(Weights1):
    W_opt1 = np.array(Weights1)
    Stdev_opt1 = np.sqrt(np.dot(W_opt1.T, np.dot(df_Cov, W_opt1)))
    return Stdev_opt1

def Rets1(Weights1):
    W_opt1 = np.array(Weights1)
    Ret = np.sum(df_AvgRet * W_opt1)
    return Ret

for Ret in returns1:
    constraints1 = ({'type':'eq', 'fun':SumIsOne1},
                   {'type':'eq', 'fun': lambda w0: Rets1(w0) - Ret})
    opt2 = minimize(minStdev1, w0, method='SLSQP', bounds=bounds1, constraints=constraints1)
    vol_opt1.append(opt2['fun'])

Vol_Opt1 = np.array(vol_opt1)
Ret1 = np.array(returns1)

Port_Eff = [Vol_Opt1, Ret1]
Port_Eff = pd.DataFrame(Port_Eff).T
Port_Eff.columns = ['Std. Dev', 'Returns']

1.0
1.0


## Fama-French/Carhart 4 Factors

In [5]:
df_ff3_monthly = gff.carhart4Factor(frequency='m')
df_ff3_monthly = df_ff3_monthly.rename(columns={"date_ff_factors": 'Date'})
df_ff3_monthly.set_index('Date', inplace=True)
Man = df[['VOX', 'VDC','VDE','VEU', 'VNQI']]
Port_Ret = Man*w
Port_Ret = Port_Ret.sum(axis=1)
Port_Ret = pd.DataFrame(Port_Ret)
Port_Ret.columns = ['Portfolio Return']
#Port_Ret = Port_Ret[1::]
FF3 = pd.merge_asof(Port_Ret['Portfolio Return'], df_ff3_monthly, on='Date')
FF3 = pd.merge_asof(FF3, df['IWV'], on='Date')

FF3

,Date,Portfolio Return,Mkt-RF,SMB,HML,RF,MOM,IWV
0,2017-10-01,0.000000,0.0251,0.0446,0.0312,0.0009,-0.0132,NaN
1,2017-11-01,0.010849,0.0225,-0.0193,0.0021,0.0009,0.0427,NaN
2,2017-12-01,0.013149,0.0312,-0.0058,-0.0008,0.0008,-0.0087,0.007042
3,2018-01-01,0.046653,0.0106,-0.0132,0.0005,0.0009,-0.0154,0.054671
4,2018-02-01,-0.075483,0.0557,-0.0315,-0.0133,0.0012,0.0405,-0.037731
...,...,...,...,...,...,...,...,...
56,2022-06-01,-0.098906,-0.0034,-0.0185,0.0841,0.0003,0.0248,-0.091082
57,2022-07-01,0.052765,-0.0843,0.0209,-0.0597,0.0006,0.0079,0.093288
58,2022-08-01,-0.024405,0.0957,0.0281,-0.0410,0.0008,-0.0396,-0.038110
59,2022-09-01,-0.114608,-0.0378,0.0139,0.0031,0.0019,0.0210,-0.101290


# Dash App

In [6]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

app.layout = html.Div([
    html.Br(),
    html.Img(src='https://www.mhskids.org/wp-content/uploads/2016/01/TRUST.jpg', 
             style={"display":"inline-block", "width":"10%", 'margin-left': '7px'}),
    html.H1('Hershey Trust Company Interview',
            style={'text-align': 'center', 'display':'inline-block', 'width':'88%'}),
    html.H4('Investment Analysis Exercise Example',
            style={'text-align': 'center', 'display':'inline-block', 'width':'100%'}),
    html.H6('Leonardo F. de Souza',
            style={'text-align': 'center', 'width':'100%'}),
    html.P('October 28th, 2022',
            style={'text-align': 'center', 'width':'100%'}),
    
    html.Hr(),
    html.Br(),
#     dcc.Interval(
#                 id='interval',
#                 disabled=False,
#                 interval=1.5*10000, #Updated every 15 seconds (this is in milliseconds)
#                 n_intervals=0,
#                 max_intervals=-1, #-1 is unilimited times of updates - if = 4, it's updated only 4 times
#     ),
    html.Div([dcc.Dropdown(id='Select_Ticker',
                               options=ticker_options,
                               value='AAPL',
                               placeholder='Ticker',
                               style={'color': 'black'}),
                  ], style={'width': '32.5%', 'text-align': 'center', 'margin-left': '7px' , "display": "inline-block"}),
    html.Div([dcc.Dropdown(id='Y_N',
                               options=['Yes', 'No'],
                               #value='No',
                               placeholder='Replace Manager (Y/N)',
                               style={'color': 'black'}),
                  ], style={'width': '32.5%', 'text-align': 'center', 'margin-left': '7px' , "display": "inline-block"}),
    html.Div([dcc.Dropdown(id='Which',
                               options=['VOX', 'VDC','VDE','VEU', 'VNQI'],
                               placeholder='Replace Manager',
                               style={'color': 'black'}),
                  ], style={'width': '32.5%', 'text-align': 'center', 'margin-left': '7px' , "display": "inline-block"}),
    html.Div([dcc.Input(id='Select_Allocation', type='number', value = 0,step=1000000, min=0,
                        placeholder='Select Allocation ($)')],
            style={'width':'50%','text-align': 'center', "display": "inline-block", 'margin-left': '7px'}), 
    ### NOTE THAT FX WON'T HAVE CALLS/PUTS TAB
    html.Br(),
    html.Br(),
    html.Div([dcc.RadioItems(['Attribution', 'BHB'], 'Attribution', id='rad')], 
             style = {'text-align': 'right', 'margin-right': '7px' }),
    html.Div([(dcc.Graph(id='chart',
                         style={'width': "100%", 'height': '120%', "display": "inline-block", 'text-align': 'center'}))],
             style = {'width':'50%','text-align': 'center', "display": "inline-block"}),
    html.Div([(
        dcc.Graph(id='chart_1',
                         style={'width': "100%", 'height': '120%', "display": "inline-block", 'text-align': 'center'}))],
             style = {'width':'50%','text-align': 'center', "display": "inline-block"}),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Tabs(id="tabs-example-graph", value='tab-1-example-graph', children=[
            dcc.Tab(label='Attribution', value='tab-1-example-graph'),
            dcc.Tab(label='Appraisal', value='tab-2-example-graph'),
            dcc.Tab(label='Current Allocation', value='tab-3-example-graph'),
            dcc.Tab(label='Fama-French', value='tab-4-example-graph'),
            dcc.Tab(label='Manager Returns', value='tab-5-example-graph'),
        ], colors={"border":"gold", "primary":"#55595c", "background":"black"}),
        html.Div(id='tabs-content-example-graph')
    ]),
    html.Br(),
    html.Hr(),
    html.I('For internal use only. Not to be distributed outside of Hershey Trust Company. Leo de Souza, October 28th, 2022'),
    html.Br(),
])

## Callbacks

In [7]:
@app.callback(
    Output("chart", "figure"),
    [Input("Select_Ticker", "value")])
def update_graph(man):
    Managers = yf.download(man, interval=interval, start=start)['Adj Close']
    df1 = pd.DataFrame(Managers)
    df1.dropna(how='all',axis=0, inplace=True)
    df1.dropna(how='all',axis=1, inplace=True)
    df1 = np.log(df1) - np.log(df1.shift(1))
    df2 = pd.merge_asof(df, df1, on='Date')
    df2 = df2.rename(columns={"Adj Close": 'New Manager'})
    test2 = df2[['VOX', 'VDC','VDE','VEU', 'VNQI', 'New Manager']]
    test2 = test2
    test2 = ((1+test2).div(1+df2.IWV, axis=0))-1
    test2 = test2[1::]
    t_cor2 = round(test2.corr(),3)
    cor_fig = px.imshow(t_cor2, text_auto=True, aspect="auto",
                    labels=dict(color="Correlation"))
    cor_fig.update_layout(
    title_text ='Correlation Matrix - Excess Returns',
    yaxis_title ='Managers')
    cor_fig.update_xaxes(side="top")
    return cor_fig

In [8]:
@app.callback(
    Output('chart_1', 'figure'),
    [Input("Select_Ticker", "value")],
    [Input("Y_N", "value")],
    [Input("Which", "value")],
    [Input("rad", "value")],
    [Input("Select_Allocation", "value")])
def update_graph1(man, yn, whi, rad, al):
    Managers = yf.download(man, interval=interval, start=start)['Adj Close']
    df1 = pd.DataFrame(Managers)
    df1.dropna(how='all',axis=0, inplace=True)
    df1.dropna(how='all',axis=1, inplace=True)
    df1 = np.log(df1) - np.log(df1.shift(1))
    df2 = pd.merge_asof(df, df1, on='Date')
    df2 = df2.rename(columns={"Adj Close": 'New Manager'})
    test2 = df2[['VOX', 'VDC','VDE','VEU', 'VNQI', 'New Manager']]
    test2 = test2
    if yn == 'Yes':
        yndf = df2[['VOX', 'VDC','VDE','VEU', 'VNQI']]
        yndf[whi] = test2['New Manager']
        yndf = yndf.rename(columns={whi: 'New Manager'})
        P = 100000000
        w2 = pd.Series([(1/(len(yndf.columns)))*P for x in range(len(yndf.columns))]) # funding of each manager
        w1 = [weight/P for weight in w2]
        AvgRet = ((np.prod(1+yndf))**(12/((yndf.count()))))-1
        if rad == 'Attribution':
            N = yndf.columns
            R1 = [N, w1, AvgRet.tolist()]
            R1 = pd.DataFrame(R1).T
            R1.columns = ['Security/Manager', 'Portfolio Weight', 'Portfolio Return']
            R1['Contribution'] = R1['Portfolio Weight']*R1['Portfolio Return']
            R1.loc[len(R1.index)] = ['Portfolio', np.sum(R1['Portfolio Weight']),
                             np.sum(R1['Portfolio Weight']*R1['Portfolio Return']), 
                             np.sum(R1['Contribution'])]
            fig = px.bar(R1, y='Contribution', x='Security/Manager',
             title='Return Contribution', color='Security/Manager')
            fig.update_layout(yaxis_tickformat=',.2%')
            return fig
        elif rad == 'BHB':
            N = yndf.columns
            R1 = [N, w1, AvgRet.tolist()]
            R1 = pd.DataFrame(R1).T
            R1.columns = ['Security/Manager', 'Portfolio Weight', 'Portfolio Return']
            BMH = pd.DataFrame({'Benchmark Weight': [0.20, 0.15, 0.10, 0.25, 0.1, 0.2, np.nan], 
                    'Benchmark Return': [-0.03, 0.06, 0.01, -0.05, -0.1, 0.015, np.nan]})
            R1.insert(loc=2, column='Benchmark Weight', value=BMH['Benchmark Weight'])
            R1.insert(loc=4, column='Benchmark Return', value=BMH['Benchmark Return'])
            R1.at[6,'Benchmark Weight']= np.sum(R1['Benchmark Weight'])
            R1.at[6,'Benchmark Return']= np.sum(R1['Benchmark Weight']*R1['Benchmark Return'])
            R1.at[6,'Security/Manager']= 'Totals'
            R1['Tactical Asset Allocation'] = (R1['Portfolio Weight'] - R1['Benchmark Weight'])*R1['Portfolio Return']
            R1['Stock Selection'] = R1['Benchmark Weight']*(R1['Portfolio Return'] - R1['Benchmark Return'])
            R1['Interaction'] = (R1['Portfolio Weight'] - R1['Benchmark Weight'])*(R1['Portfolio Return'] - R1['Benchmark Return'])
            R1['Total Value Added'] = (R1['Portfolio Weight']*R1['Portfolio Return'])-(R1['Benchmark Weight']*R1['Benchmark Return'])
            fig = make_subplots(rows=2, cols=2, subplot_titles=("Tactical Asset Allocation", "Stock Selection", 
                                                    "Interaction", "Total Value Added"))
            fig.add_trace(go.Waterfall(
                #name = "20",
                orientation = "v",
                measure = ['relative' for x in R1['Security/Manager'][:-1:]]+['total'],
                x = R1['Security/Manager'],
                #textposition = "outside",
                #text = R1['Tactical Asset Allocation'],
                y = R1['Tactical Asset Allocation'],
                connector = {"line":{"color":"rgb(63, 63, 63)"}}),row=1, col=1)
            fig.add_trace(go.Waterfall(
                #name = "20",
                orientation = "v",
                measure = ['relative' for x in R1['Security/Manager'][:-1:]]+['total'],
                x = R1['Security/Manager'],
                #textposition = "outside",
                #text = R1['Stock Selection'],
                y = R1['Stock Selection'],
                connector = {"line":{"color":"rgb(63, 63, 63)"}}),row=1, col=2)
            fig.add_trace(go.Waterfall(
                #name = "20",
                orientation = "v",
                measure = ['relative' for x in R1['Security/Manager'][:-1:]]+['total'],
                x = R1['Security/Manager'],
                #textposition = "outside",
                #text = R1['Interaction'],
                y = R1['Interaction'],
                connector = {"line":{"color":"rgb(63, 63, 63)"}}),row=2, col=1)

            fig.add_trace(go.Waterfall(
                #name = "20",
                orientation = "v",
                measure = ['relative' for x in R1['Security/Manager'][:-1:]]+['total'],
                x = R1['Security/Manager'],
                #textposition = "outside",
                #text = R1['Total Value Added'],
                y = R1['Total Value Added'],
                connector = {"line":{"color":"rgb(63, 63, 63)"}}),row=2, col=2)
            fig.update_layout(title_text="BHB Model***")
            fig.update_layout(showlegend=False)
            return fig
    else:
        P = 100000000
        wNM = [al/(al+P)]
        w1 = [weight * (P/(al+P)) for weight in w]
        W = w1+wNM
        AvgRet = ((np.prod(1+test2))**(12/((test2.count()))))-1
        if rad == 'Attribution':
            N = test2.columns
            R1 = [N, W, AvgRet.tolist()]
            R1 = pd.DataFrame(R1).T
            R1.columns = ['Security/Manager', 'Portfolio Weight', 'Portfolio Return']
            R1['Contribution'] = R1['Portfolio Weight']*R1['Portfolio Return']
            R1.loc[len(R1.index)] = ['Portfolio', np.sum(R1['Portfolio Weight']),
                             np.sum(R1['Portfolio Weight']*R1['Portfolio Return']), 
                             np.sum(R1['Contribution'])]
            fig = px.bar(R1, y='Contribution', x='Security/Manager',
                         title='Return Contribution', color='Security/Manager')
            fig.update_layout(yaxis_tickformat=',.2%')
            return fig
        elif rad == 'BHB':
            N = test2.columns
            R1 = [N, W, AvgRet.tolist()]
            R1 = pd.DataFrame(R1).T
            R1.columns = ['Security/Manager', 'Portfolio Weight', 'Portfolio Return']
            BMH = pd.DataFrame({'Benchmark Weight': [0.20, 0.15, 0.10, 0.25, 0.1, 0.2, np.nan], 
                    'Benchmark Return': [-0.03, 0.06, 0.01, -0.05, -0.1, 0.015, np.nan]})
            R1.insert(loc=2, column='Benchmark Weight', value=BMH['Benchmark Weight'])
            R1.insert(loc=4, column='Benchmark Return', value=BMH['Benchmark Return'])
            R1.at[6,'Benchmark Weight']= np.sum(R1['Benchmark Weight'])
            R1.at[6,'Benchmark Return']= np.sum(R1['Benchmark Weight']*R1['Benchmark Return'])
            R1.at[6,'Security/Manager']= 'Totals'
            R1['Tactical Asset Allocation'] = (R1['Portfolio Weight'] - R1['Benchmark Weight'])*R1['Portfolio Return']
            R1['Stock Selection'] = R1['Benchmark Weight']*(R1['Portfolio Return'] - R1['Benchmark Return'])
            R1['Interaction'] = (R1['Portfolio Weight'] - R1['Benchmark Weight'])*(R1['Portfolio Return'] - R1['Benchmark Return'])
            R1['Total Value Added'] = (R1['Portfolio Weight']*R1['Portfolio Return'])-(R1['Benchmark Weight']*R1['Benchmark Return'])
            fig = make_subplots(rows=2, cols=2, subplot_titles=("Tactical Asset Allocation", "Stock Selection", 
                                                    "Interaction", "Total Value Added"))
            fig.add_trace(go.Waterfall(
                #name = "20",
                orientation = "v",
                measure = ['relative' for x in R1['Security/Manager'][:-1:]]+['total'],
                x = R1['Security/Manager'],
                #textposition = "outside",
                #text = R1['Tactical Asset Allocation'],
                y = R1['Tactical Asset Allocation'],
                connector = {"line":{"color":"rgb(63, 63, 63)"}}),row=1, col=1)
            fig.add_trace(go.Waterfall(
                #name = "20",
                orientation = "v",
                measure = ['relative' for x in R1['Security/Manager'][:-1:]]+['total'],
                x = R1['Security/Manager'],
                #textposition = "outside",
                #text = R1['Stock Selection'],
                y = R1['Stock Selection'],
                connector = {"line":{"color":"rgb(63, 63, 63)"}}),row=1, col=2)
            fig.add_trace(go.Waterfall(
                #name = "20",
                orientation = "v",
                measure = ['relative' for x in R1['Security/Manager'][:-1:]]+['total'],
                x = R1['Security/Manager'],
                #textposition = "outside",
                #text = R1['Interaction'],
                y = R1['Interaction'],
                connector = {"line":{"color":"rgb(63, 63, 63)"}}),row=2, col=1)

            fig.add_trace(go.Waterfall(
                #name = "20",
                orientation = "v",
                measure = ['relative' for x in R1['Security/Manager'][:-1:]]+['total'],
                x = R1['Security/Manager'],
                #textposition = "outside",
                #text = R1['Total Value Added'],
                y = R1['Total Value Added'],
                connector = {"line":{"color":"rgb(63, 63, 63)"}}),row=2, col=2)
            fig.update_layout(title_text="BHB Model***")
            fig.update_layout(showlegend=False)
            return fig

In [9]:
@app.callback(Output('tabs-content-example-graph', 'children'),
              [Input("Select_Ticker", "value")],
              [Input("Y_N", "value")],
              [Input("Which", "value")],
              [Input("Select_Allocation", "value")],
              [Input('tabs-example-graph', 'value')])
def update_graph1(man, yn, whi, al, tab):
    Managers = yf.download(man, interval=interval, start=start)['Adj Close']
    df1 = pd.DataFrame(Managers)
    df1.dropna(how='all',axis=0, inplace=True)
    df1.dropna(how='all',axis=1, inplace=True)
    df1 = np.log(df1) - np.log(df1.shift(1))
    df2 = pd.merge_asof(df, df1, on='Date')
    df2 = df2.rename(columns={"Adj Close": 'New Manager'})
    test2 = df2[['VOX', 'VDC','VDE','VEU', 'VNQI', 'New Manager']]
    test2 = test2
    Man = df2[['VOX', 'VDC','VDE','VEU', 'VNQI']]
    Port_Ret = Man*w
    Port_Ret = Port_Ret.sum(axis=1)
    Port_Ret = pd.DataFrame(Port_Ret)
    Port_Ret.columns = ['Portfolio Return']
    Port_Ret = Port_Ret[1::]
    Excessa = ((1+Port_Ret['Portfolio Return'])/(1+df2['IWV']))-1
    Tracking_Error_1 = Excessa.std()*np.sqrt(12)
    Annualized_Return_1 = ((np.prod(1+Port_Ret['Portfolio Return']))**(12/((Port_Ret['Portfolio Return'].count()))))-1
    Stdev_1 = (Port_Ret['Portfolio Return'].std()*np.sqrt(12))
    BMR = ((np.prod(1+df['IWV']))**(12/((df['IWV'].count()))))-1
    IR = (Annualized_Return_1-BMR)/(Tracking_Error_1)
    SP = (Annualized_Return_1-BMR)/(Stdev_1)
    Risk_Reward = Annualized_Return_1/Stdev_1
    y = Port_Ret['Portfolio Return']
    x = df2['New Manager'] #CHANGE
    x = x[1::]
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    Beta_Adj = slope*x
    ERB = ((1+Beta_Adj)/(1+df2['IWV'][1::]))-1
    TEB = ERB.std()*np.sqrt(12)
    ARB = ((np.prod(1+Beta_Adj))**(12/((Beta_Adj.count()))))-1
    SDB = (Beta_Adj.std()*np.sqrt(12))
    IRB = (ARB-BMR)/(TEB)
    RRB = ARB/SDB
    y1 = Port_Ret['Portfolio Return']
    x1 = Beta_Adj
    slope1, intercept1, r_value1, p_value1, std_err1 = linregress(x1, y1)
    APP1 = (intercept1/std_err1)

    #2. Unadjusted:
    ERB2 = ((1+x)/(1+df2['IWV'][1::]))-1
    TEB2 = ERB2.std()*np.sqrt(12)
    ARB2 = ((np.prod(1+x))**(12/((x.count()))))-1
    SDB2 = (x.std()*np.sqrt(12))
    IRB2 = (ARB2-BMR)/(TEB2)
    RRB2 = ARB2/SDB2
    APP2 = (intercept/std_err)

    Comp1 = ["Annualized Returns", "Standard Deviation", "Information Ratio", "Reward/Risk"]
    Comp1 = pd.DataFrame(Comp1)
    Comp1['Original Portfolio'] = [Annualized_Return_1, Stdev_1, IR, Risk_Reward]
    Comp1['Beta Adj. Manager'] = [ARB, SDB, IRB, RRB]
    Comp1['Beta Unadj. Manager'] = [ARB2, SDB2, IRB2, RRB2]
    Comp1 = Comp1.rename(columns = {Comp1.columns[0]: "Metrics"})
    Comp1.loc[len(Comp1.index)] = ['Appraisal Ratio', "-", APP1, APP2] 
    
    if yn == 'Yes':
        yndf = df2[['VOX', 'VDC','VDE','VEU', 'VNQI']]
        yndf[whi] = test2['New Manager']
        yndf = yndf.rename(columns={whi: 'New Manager'})
        P = 100000000
        w2 = pd.Series([(1/(len(yndf.columns)))*P for x in range(len(yndf.columns))]) # funding of each manager
        w1 = [weight/P for weight in w2]
        Port_Ret2 = yndf*w1
        Port_Ret2 = Port_Ret2.sum(axis=1)
        Port_Ret2 = pd.DataFrame(Port_Ret2)
        Port_Ret2.columns = ['Portfolio Return']
        Port_Ret2 = Port_Ret2[1::]
        Excess3 = ((1+Port_Ret2['Portfolio Return'])/(1+df2['IWV']))-1
        Tracking_Error_3 = Excess3.std()*np.sqrt(12)
        Annualized_Return_3 = ((np.prod(1+Port_Ret2['Portfolio Return']))**(12/((Port_Ret2['Portfolio Return'].count()))))-1
        Stdev_3 = (Port_Ret2['Portfolio Return'].std()*np.sqrt(12))
        BMR = ((np.prod(1+df2['IWV']))**(12/((df2['IWV'].count()))))-1
        IR3 = (Annualized_Return_3-BMR)/(Tracking_Error_3)
        Risk_Reward3 = Annualized_Return_3/Stdev_3
        if tab == 'tab-1-example-graph':
            Comp3 = ["Annualized Returns", "Standard Deviation", "Information Ratio", "Reward/Risk", "Tracking Error"]
            Comp3 = pd.DataFrame(Comp3)
            Comp3['Original Portfolio'] = [Annualized_Return_1, Stdev_1, IR, Risk_Reward, Tracking_Error_1]
            Comp3['New Portfolio'] = [Annualized_Return_3, Stdev_3, IR3, Risk_Reward3, Tracking_Error_3]
            Comp3['Changes'] = Comp3['New Portfolio'] - Comp3['Original Portfolio']
            Comp3 = Comp3.rename(columns={Comp3.columns[0]: "Metrics"})
            columns = [{"name": i, 'type': 'numeric', 'format': {'specifier': '.4%'}, "id": i} for i
                       in Comp3.columns]
            lab = Comp3.to_dict('records')
            return html.Div([
                html.Br(),
                html.H6('Attribution'),
                html.Br(),
                html.Div([dash_table.DataTable(data=lab, columns=columns,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })], 
                         style={'width':'75%','text-align': 'center', 'display': 'inline-block'})
                            ],style={'width':'100%','text-align': 'center', 'margin-left': '7px', 'margin-right':'7px'})
        elif tab == 'tab-2-example-graph':
            columns = [{"name": i, 'type': 'numeric', 'format': {'specifier': '.4%'}, "id": i} for i
                       in Comp1.columns]
            lab = Comp1.to_dict('records')
            return html.Div([
                html.Br(),
                html.H6('Appraisal'),
                html.Br(),
                html.Div([dash_table.DataTable(data=lab, columns=columns,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })], 
                         style={'width':'75%','text-align': 'center', 'display': 'inline-block'})
                            ],style={'width':'100%','text-align': 'center', 'margin-left': '7px', 'margin-right':'7px'})
        
        elif tab == 'tab-3-example-graph':
            W = w
            N = df2[['VOX', 'VDC','VDE','VEU', 'VNQI']].columns
            Weights = [W, N]
            Weights = pd.DataFrame(Weights).T
            Weights.columns = ['Weight', 'Security']
            
            ww = df2[['VOX', 'VDC','VDE','VEU', 'VNQI']]
            ww[whi] = test2['New Manager']
            ww = ww.rename(columns={whi: 'New Manager'})
            P = 100000000
            ww2 = pd.Series([(1/(len(ww.columns)))*P for x in range(len(ww.columns))]) # funding of each manager
            ww1 = [weight/P for weight in ww2]
            NW = ww.columns
            WW = ww1
            W_1 = [WW, NW]
            W_1 = pd.DataFrame(W_1).T
            W_1.columns = ['a', 'Security']
            
            newdf = Weights.merge(W_1, how='outer')
            newdf = newdf.replace(np.nan, 0)
            newdf['Δ Weight'] = newdf['a'] - newdf['Weight']   
            
            Comp3 = ["Annualized Returns", "Standard Deviation"]
            Comp3 = pd.DataFrame(Comp3)
            Comp3['New Portfolio'] = [Annualized_Return_3, Stdev_3]
            Comp3['Original Portfolio'] = [Annualized_Return_1, Stdev_1]
            Comp3 = Comp3.rename(columns={Comp3.columns[0]: "Metrics"})
            figure = go.Figure()
            figure.add_trace(go.Scatter(x=Port_Eff['Std. Dev'], y=Port_Eff['Returns'], name='Efficient Frontier'))
            figure.add_trace(go.Scatter(x=MV_Port['Std. Dev'], y=MV_Port['Returns'], name='Minimum Variance'))
            figure.add_trace(go.Scatter(x=S_Port['Std. Dev'], y=S_Port['Returns'], name='Tangent Portfolio'))
            figure.add_trace(go.Scatter(x=Comp3['New Portfolio'].tail(1), y=Comp3['New Portfolio'].head(1), name='New Portfolio'))
            figure.add_trace(go.Scatter(x=Comp3['Original Portfolio'].tail(1), y=Comp3['Original Portfolio'].head(1), name='Current Portfolio'))
            figure.update_layout(yaxis_tickformat=',.2%', xaxis_tickformat=',.2%', title_text ='Portfolio')
            figure.update_layout(
                          title_text ='Original Efficient Frontier vs. New Allocation',
                          yaxis_title ='Returns (μ)',
                          xaxis_title = 'Standard Deviation (σ)')
            fig = px.pie(newdf, values='a', names='Security', labels={'Security':'Security'},
                         title='New Allocation')
            fig.update_traces(textposition='inside', textinfo='percent+label')
            fig.update_layout(yaxis_tickformat=',.2%')
            
            fig2 = px.bar(newdf, y='Security', x='Δ Weight', color='Security', orientation='h',
                         title='Weight Changes')
            fig2.update_layout(xaxis_tickformat=',.2%')
            fig2.update_layout(xaxis=dict(showgrid=False))
            fig2.update_layout(showlegend=False)
            return html.Div([
                html.Div([dcc.Graph(figure = figure)], style={'width': '49%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                html.Div([dcc.Graph(figure = fig)], style={'width': '24%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                html.Div([dcc.Graph(figure = fig2)], style={'width': '24%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                            ])
        elif tab == 'tab-4-example-graph':
            FF31 = FF3[1::]
            FF31['P-rf'] = ((1+FF31['Portfolio Return'])/(1+ FF31['RF']))-1
            FF31['BM-rf'] = ((1+FF31['IWV'])/(1+ FF31['RF']))-1
            X = FF31[['BM-rf', 'SMB', 'HML', 'MOM']][3::]
            y = FF31['P-rf'][3::]
            X = sm.add_constant(X)
            ff_model = sm.OLS(y, X).fit()
            ff_model = ff_model.summary()
            results_as_html = ff_model.tables[1].as_html()
            fam = pd.read_html(results_as_html, header=0, index_col=0)[0]
            fam.insert(loc=0, column=' ', value=['Constant', 'BM-rf', 'SMB', 'HML', 'MOM'])
            results_as_html1 = ff_model.tables[0].as_html()
            Rsq = pd.read_html(results_as_html1, header=0, index_col=0)[0]
            Rsq.insert(loc=0, column=" ", value=['Model:', 'Method:', 'Date:', 'Time:', 'No. Observations:',
                                                 'Df Residuals:', 'Df Model:', 'Covariance Type:'])
            columns = [{"name": i, "id": i} for i
                       in fam.columns]
            lab = fam.to_dict('records')
            col1 =[{"name": i, "id": i} for i
                       in Rsq.columns]
            lab1 = Rsq.to_dict('records')
            
            ##
            y1 = (((1+Port_Ret2['Portfolio Return'])/(1+FF31['RF']))-1)[3::]
            ff_model2 = sm.OLS(y1, X).fit()
            ff_model2 = ff_model2.summary()
            
            results_as_html2 = ff_model2.tables[1].as_html()
            fam2 = pd.read_html(results_as_html2, header=0, index_col=0)[0]
            fam2.insert(loc=0, column=' ', value=['Constant', 'BM-rf', 'SMB', 'HML', 'MOM'])
            
            results_as_html22 = ff_model2.tables[0].as_html()
            Rsq2 = pd.read_html(results_as_html22, header=0, index_col=0)[0]
            Rsq2.insert(loc=0, column=" ", value=['Model:', 'Method:', 'Date:', 'Time:', 'No. Observations:',
                                                 'Df Residuals:', 'Df Model:', 'Covariance Type:'])
            columns2 = [{"name": i, "id": i} for i
                       in fam2.columns]
            lab22 = fam2.to_dict('records')
            
            col2 =[{"name": i, "id": i} for i
                       in Rsq2.columns]
            lab2 = Rsq2.to_dict('records')
            
            ##
            y2 = (((1+(test2['New Manager'][1::]))/(1+FF31['RF']))-1)[3::]
            ff_model3 = sm.OLS(y2, X).fit()
            ff_model3 = ff_model3.summary()
            
            results_as_html3 = ff_model3.tables[1].as_html()
            fam3 = pd.read_html(results_as_html3, header=0, index_col=0)[0]
            fam3.insert(loc=0, column=' ', value=['Constant', 'BM-rf', 'SMB', 'HML', 'MOM'])
            
            results_as_html33 = ff_model3.tables[0].as_html()
            Rsq3 = pd.read_html(results_as_html33, header=0, index_col=0)[0]
            Rsq3.insert(loc=0, column=" ", value=['Model:', 'Method:', 'Date:', 'Time:', 'No. Observations:',
                                                 'Df Residuals:', 'Df Model:', 'Covariance Type:'])
            columns3 = [{"name": i, "id": i} for i
                       in fam3.columns]
            lab33 = fam3.to_dict('records')
            col3 =[{"name": i, "id": i} for i
                       in Rsq3.columns]
            lab3 = Rsq3.to_dict('records')
            
            return html.Div([
                            html.Div([
                                html.Br(),
                                html.H6('Current Portfolio'),
                                html.Br(),
                                html.Div([dash_table.DataTable(data=lab1, columns=col1,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })]),
                
                            html.Div([dash_table.DataTable(data=lab, columns=columns,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })])], 
                                style={'width': '32.5%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                            html.Div([
                                html.Br(),
                                html.H6('New Portfolio'),
                                html.Br(),
                                html.Div([dash_table.DataTable(data=lab2, columns=col2,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })]),
                
                            html.Div([dash_table.DataTable(data=lab22, columns=columns2,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })])], 
                                style={'width': '32.5%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                            html.Div([
                                html.Br(),
                                html.H6('New Manager'),
                                html.Br(),
                                html.Div([dash_table.DataTable(data=lab3, columns=col3,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })]),
                
                            html.Div([dash_table.DataTable(data=lab33, columns=columns3,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })])], 
                                style={'width': '32.5%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                            html.P('* All output is based on a 5 year horizon.')
                            ])
        
        elif tab == 'tab-5-example-graph':
            df2['3Y'] = df2['New Manager'].rolling(3*12).mean()
            df2['1Y'] = df2['New Manager'].rolling(1*12).mean()
            Port_Ret['PR3'] = Port_Ret['Portfolio Return'].rolling(3*12).mean()
            Port_Ret['PR1'] = Port_Ret['Portfolio Return'].rolling(1*12).mean()
            Port_Ret2['PR3'] = Port_Ret2['Portfolio Return'].rolling(3*12).mean()
            Port_Ret2['PR1'] = Port_Ret2['Portfolio Return'].rolling(1*12).mean()
            figure = go.Figure()
            figure.add_trace(go.Scatter(x=FF3['Date'][1::], y=df2['New Manager'], name='New Manager'))
            figure.add_trace(go.Scatter(x=FF3['Date'][1::], y=df2['IWV'], name='Benchmark'))
            figure.add_trace(go.Scatter(x=FF3['Date'][1::], y=Port_Ret['Portfolio Return'].shift(1), name='Current Portfolio'))
            figure.add_trace(go.Scatter(x=FF3['Date'][1::], y=Port_Ret2['Portfolio Return'].shift(1), name='New Portfolio'))
            figure.update_layout(
                    title_text ='Month-to-Month Returns',
                    yaxis_title ='Returns',
                    xaxis_title = 'Date')
            figure.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=5,
                            label="5Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=3,
                            label="3Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=1,
                            label="YTD",
                            step="year",
                            stepmode="todate"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True,
                ),
                type="date"
                )
            )
            figure.update_layout(yaxis_tickformat='.2%')
            figure.update_layout(template='plotly_dark',
                  xaxis_rangeselector_font_color='black',
                  xaxis_rangeselector_activecolor='red',
                  xaxis_rangeselector_bgcolor='green',)
            figure.update_layout(xaxis=dict(showgrid=False),
                     yaxis=dict(showgrid=False))
            
            figure1 = go.Figure()
            figure1.add_trace(go.Scatter(x=FF3['Date'][12::], y=df2['1Y'], name='Manager'))
            figure1.add_trace(go.Scatter(x=FF3['Date'][12::], y=Port_Ret['PR1'].shift(1), name='Current Portfolio'))
            figure1.add_trace(go.Scatter(x=FF3['Date'][12::], y=Port_Ret2['PR1'].shift(1), name='New Portfolio'))
            figure1.update_layout(
                    title_text ='1 Year Rolling Returns',
                    yaxis_title ='Returns',
                    xaxis_title = 'Date')
            figure1.update_layout(
                xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=5,
                            label="5Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=3,
                            label="3Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=1,
                            label="YTD",
                            step="year",
                            stepmode="todate"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True,
                ),
                type="date"
                )
            )
            figure1.update_layout(yaxis_tickformat='.2%')
            figure1.update_layout(template='plotly_dark',
                  xaxis_rangeselector_font_color='black',
                  xaxis_rangeselector_activecolor='red',
                  xaxis_rangeselector_bgcolor='green',)
            figure1.update_layout(xaxis=dict(showgrid=False),
                     yaxis=dict(showgrid=False))
            
        return html.Div([
            html.Div([
            dcc.Graph(figure = figure)], style={'width': '49%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
            html.Div([dcc.Graph(figure = figure1)], style={'width': '49%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"})])
            
    else:
        P = 100000000
        wNM = [al/(al+P)]
        w1 = [weight * (P/(al+P)) for weight in w]
        W = w1+wNM
        Port_Ret2 = test2*W
        Port_Ret2 = Port_Ret2.sum(axis=1)
        Port_Ret2 = pd.DataFrame(Port_Ret2)
        Port_Ret2.columns = ['Portfolio Return']
        Port_Ret2 = Port_Ret2[1::]
        Excess3 = ((1+Port_Ret2['Portfolio Return'])/(1+df2['IWV']))-1
        Tracking_Error_3 = Excess3.std()*np.sqrt(12)
        Annualized_Return_3 = ((np.prod(1+Port_Ret2['Portfolio Return']))**(12/((Port_Ret2['Portfolio Return'].count()))))-1
        Stdev_3 = (Port_Ret2['Portfolio Return'].std()*np.sqrt(12))
        BMR = ((np.prod(1+df2['IWV']))**(12/((df2['IWV'].count()))))-1
        IR3 = (Annualized_Return_3-BMR)/(Tracking_Error_3)
        Risk_Reward3 = Annualized_Return_3/Stdev_3
        if tab == 'tab-1-example-graph':
            Comp3 = ["Annualized Returns", "Standard Deviation", "Information Ratio", "Reward/Risk", "Tracking Error"]
            Comp3 = pd.DataFrame(Comp3)
            Comp3['Original Portfolio'] = [Annualized_Return_1, Stdev_1, IR, Risk_Reward, Tracking_Error_1]
            Comp3['New Portfolio'] = [Annualized_Return_3, Stdev_3, IR3, Risk_Reward3, Tracking_Error_3]
            Comp3['Changes'] = Comp3['New Portfolio'] - Comp3['Original Portfolio']
            Comp3 = Comp3.rename(columns={Comp3.columns[0]: "Metrics"})
            columns = [{"name": i, 'type': 'numeric', 'format': {'specifier': '.4%'}, "id": i} for i
                       in Comp3.columns]
            lab = Comp3.to_dict('records')
            return html.Div([
                html.Br(),
                html.H6('Attribution'),
                html.Br(),
                html.Div([dash_table.DataTable(data=lab, columns=columns,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })], 
                         style={'width':'75%','text-align': 'center', 'display': 'inline-block'})
                            ],style={'width':'100%','text-align': 'center', 'margin-left': '7px', 'margin-right':'7px'})
        elif tab == 'tab-2-example-graph':
            columns = [{"name": i, 'type': 'numeric', 'format': {'specifier': '.4%'}, "id": i} for i
                       in Comp1.columns]
            lab = Comp1.to_dict('records')
            
            return html.Div([
                html.Br(),
                html.H6('Appraisal'),
                html.Br(),
                html.Div([dash_table.DataTable(data=lab, columns=columns,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })], 
                         style={'width':'75%','text-align': 'center', 'display': 'inline-block'})
                            ],style={'width':'100%','text-align': 'center', 'margin-left': '7px', 'margin-right':'7px'})
        elif tab == 'tab-3-example-graph':
            W = W
            N = test2.columns
            W1 = w+[0]
            Weights = [W, N, W1]
            Weights = pd.DataFrame(Weights).T
            Weights.columns = ['Weight', 'Security', 'a']
            Weights['Δ Weight'] = Weights['Weight'] - Weights['a']
            
            
            Comp3 = ["Annualized Returns", "Standard Deviation"]
            Comp3 = pd.DataFrame(Comp3)
            Comp3['New Portfolio'] = [Annualized_Return_3, Stdev_3]
            Comp3['Original Portfolio'] = [Annualized_Return_1, Stdev_1]
            Comp3 = Comp3.rename(columns={Comp3.columns[0]: "Metrics"})
            
            figure = go.Figure()
            figure.add_trace(go.Scatter(x=Port_Eff['Std. Dev'], y=Port_Eff['Returns'], name='Efficient Frontier'))
            figure.add_trace(go.Scatter(x=MV_Port['Std. Dev'], y=MV_Port['Returns'], name='Minimum Variance'))
            figure.add_trace(go.Scatter(x=S_Port['Std. Dev'], y=S_Port['Returns'], name='Tangent Portfolio'))
            figure.add_trace(go.Scatter(x=Comp3['New Portfolio'].tail(1), y=Comp3['New Portfolio'].head(1), name='New Portfolio'))
            figure.add_trace(go.Scatter(x=Comp3['Original Portfolio'].tail(1), y=Comp3['Original Portfolio'].head(1), name='Current Portfolio'))
            figure.update_layout(yaxis_tickformat=',.2%', xaxis_tickformat=',.2%', title_text ='Portfolio')
            figure.update_layout(
                          title_text ='Original Efficient Frontier vs. New Allocation',
                          yaxis_title ='Returns (μ)',
                          xaxis_title = 'Standard Deviation (σ)')
            
            fig = px.pie(Weights, values='Weight', names='Security', labels={'Security':'Security'},
                         title='New Allocation')
            fig.update_traces(textposition='inside', textinfo='percent+label')
            fig.update_layout(yaxis_tickformat=',.2%')
            
            fig2 = px.bar(Weights, y='Security', x='Δ Weight', color='Security', orientation='h',
                         title='Weight Changes')
            fig2.update_layout(xaxis_tickformat=',.2%')
            fig2.update_layout(xaxis=dict(showgrid=False))
            fig2.update_layout(showlegend=False)
            return html.Div([
                html.Div([dcc.Graph(figure = figure)], style={'width': '49%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                html.Div([dcc.Graph(figure = fig)], style={'width': '24%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                html.Div([dcc.Graph(figure = fig2)], style={'width': '24%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                            ])
        elif tab == 'tab-4-example-graph':
            FF31 = FF3[1::]
            FF31['P-rf'] = ((1+FF31['Portfolio Return'])/(1+ FF31['RF']))-1
            FF31['BM-rf'] = ((1+FF31['IWV'])/(1+ FF31['RF']))-1
            X = FF31[['BM-rf', 'SMB', 'HML', 'MOM']][3::]
            y = FF31['P-rf'][3::]
            X = sm.add_constant(X)
            ff_model = sm.OLS(y, X).fit()
            ff_model = ff_model.summary()
            results_as_html = ff_model.tables[1].as_html()
            fam = pd.read_html(results_as_html, header=0, index_col=0)[0]
            fam.insert(loc=0, column=' ', value=['Constant', 'BM-rf', 'SMB', 'HML', 'MOM'])
            results_as_html1 = ff_model.tables[0].as_html()
            Rsq = pd.read_html(results_as_html1, header=0, index_col=0)[0]
            Rsq.insert(loc=0, column=" ", value=['Model:', 'Method:', 'Date:', 'Time:', 'No. Observations:',
                                                 'Df Residuals:', 'Df Model:', 'Covariance Type:'])
            columns = [{"name": i, "id": i} for i
                       in fam.columns]
            lab = fam.to_dict('records')
            col1 =[{"name": i, "id": i} for i
                       in Rsq.columns]
            lab1 = Rsq.to_dict('records')
            
            ##
            y1 = (((1+Port_Ret2['Portfolio Return'])/(1+FF31['RF']))-1)[3::]
            ff_model2 = sm.OLS(y1, X).fit()
            ff_model2 = ff_model2.summary()
            
            results_as_html2 = ff_model2.tables[1].as_html()
            fam2 = pd.read_html(results_as_html2, header=0, index_col=0)[0]
            fam2.insert(loc=0, column=' ', value=['Constant', 'BM-rf', 'SMB', 'HML', 'MOM'])
            
            results_as_html22 = ff_model2.tables[0].as_html()
            Rsq2 = pd.read_html(results_as_html22, header=0, index_col=0)[0]
            Rsq2.insert(loc=0, column=" ", value=['Model:', 'Method:', 'Date:', 'Time:', 'No. Observations:',
                                                 'Df Residuals:', 'Df Model:', 'Covariance Type:'])
            columns2 = [{"name": i, "id": i} for i
                       in fam2.columns]
            lab22 = fam2.to_dict('records')
            
            col2 =[{"name": i, "id": i} for i
                       in Rsq2.columns]
            lab2 = Rsq2.to_dict('records')
            
            ##
            y2 = (((1+(test2['New Manager'][1::]))/(1+FF31['RF']))-1)[3::]
            ff_model3 = sm.OLS(y2, X).fit()
            ff_model3 = ff_model3.summary()
            
            results_as_html3 = ff_model3.tables[1].as_html()
            fam3 = pd.read_html(results_as_html3, header=0, index_col=0)[0]
            fam3.insert(loc=0, column=' ', value=['Constant', 'BM-rf', 'SMB', 'HML', 'MOM'])
            
            results_as_html33 = ff_model3.tables[0].as_html()
            Rsq3 = pd.read_html(results_as_html33, header=0, index_col=0)[0]
            Rsq3.insert(loc=0, column=" ", value=['Model:', 'Method:', 'Date:', 'Time:', 'No. Observations:',
                                                 'Df Residuals:', 'Df Model:', 'Covariance Type:'])
            columns3 = [{"name": i, "id": i} for i
                       in fam3.columns]
            lab33 = fam3.to_dict('records')
            col3 =[{"name": i, "id": i} for i
                       in Rsq3.columns]
            lab3 = Rsq3.to_dict('records')
            
            return html.Div([
                            html.Div([
                                html.Br(),
                                html.H6('Current Portfolio'),
                                html.Br(),
                                html.Div([dash_table.DataTable(data=lab1, columns=col1,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })]),
                
                            html.Div([dash_table.DataTable(data=lab, columns=columns,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })])], 
                                style={'width': '32.5%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                            html.Div([
                                html.Br(),
                                html.H6('New Portfolio'),
                                html.Br(),
                                html.Div([dash_table.DataTable(data=lab2, columns=col2,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })]),
                
                            html.Div([dash_table.DataTable(data=lab22, columns=columns2,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })])], 
                                style={'width': '32.5%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                            html.Div([
                                html.Br(),
                                html.H6('New Manager'),
                                html.Br(),
                                html.Div([dash_table.DataTable(data=lab3, columns=col3,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })]),
                
                            html.Div([dash_table.DataTable(data=lab33, columns=columns3,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })])], 
                                style={'width': '32.5%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
                        html.P('* All output is based on a 5 year horizon.')
                            ])
        
        elif tab == 'tab-5-example-graph':
            df2['3Y'] = df2['New Manager'].rolling(3*12).mean()
            df2['1Y'] = df2['New Manager'].rolling(1*12).mean()
            Port_Ret['PR3'] = Port_Ret['Portfolio Return'].rolling(3*12).mean()
            Port_Ret['PR1'] = Port_Ret['Portfolio Return'].rolling(1*12).mean()
            Port_Ret2['PR3'] = Port_Ret2['Portfolio Return'].rolling(3*12).mean()
            Port_Ret2['PR1'] = Port_Ret2['Portfolio Return'].rolling(1*12).mean()
            figure = go.Figure()
            figure.add_trace(go.Scatter(x=FF3['Date'][1::], y=df2['New Manager'], name='New Manager'))
            figure.add_trace(go.Scatter(x=FF3['Date'][1::], y=df2['IWV'], name='Benchmark'))
            figure.add_trace(go.Scatter(x=FF3['Date'][1::], y=Port_Ret['Portfolio Return'].shift(1), name='Current Portfolio'))
            figure.add_trace(go.Scatter(x=FF3['Date'][1::], y=Port_Ret2['Portfolio Return'].shift(1), name='New Portfolio'))
            figure.update_layout(
                    title_text ='Month-to-Month Returns',
                    yaxis_title ='Returns',
                    xaxis_title = 'Date')
            figure.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=5,
                            label="5Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=3,
                            label="3Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=1,
                            label="YTD",
                            step="year",
                            stepmode="todate"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True,
                ),
                type="date"
                )
            )
            figure.update_layout(yaxis_tickformat='.2%')
            figure.update_layout(template='plotly_dark',
                  xaxis_rangeselector_font_color='black',
                  xaxis_rangeselector_activecolor='red',
                  xaxis_rangeselector_bgcolor='green',)
            figure.update_layout(xaxis=dict(showgrid=False),
                     yaxis=dict(showgrid=False))
            
            figure1 = go.Figure()
            figure1.add_trace(go.Scatter(x=FF3['Date'][12::], y=df2['1Y'], name='Manager'))
            figure1.add_trace(go.Scatter(x=FF3['Date'][12::], y=Port_Ret['PR1'].shift(1), name='Current Portfolio'))
            figure1.add_trace(go.Scatter(x=FF3['Date'][12::], y=Port_Ret2['PR1'].shift(1), name='New Portfolio'))
            figure1.update_layout(
                    title_text ='1 Year Rolling Returns',
                    yaxis_title ='Returns',
                    xaxis_title = 'Date')
            figure1.update_layout(
                xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=5,
                            label="5Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=3,
                            label="3Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=1,
                            label="YTD",
                            step="year",
                            stepmode="todate"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True,
                ),
                type="date"
                )
            )
            figure1.update_layout(yaxis_tickformat='.2%')
            figure1.update_layout(template='plotly_dark',
                  xaxis_rangeselector_font_color='black',
                  xaxis_rangeselector_activecolor='red',
                  xaxis_rangeselector_bgcolor='green',)
            figure1.update_layout(xaxis=dict(showgrid=False),
                     yaxis=dict(showgrid=False))
            
        return html.Div([
            html.Div([
            dcc.Graph(figure = figure)], style={'width': '49%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"}),
            html.Div([dcc.Graph(figure = figure1)], style={'width': '49%', 'text-align': 'center', 
                                       'margin-left': '7px' , "display": "inline-block"})])
        
        
            

# Run App

In [10]:
if __name__ == '__main__':
    app.run(debug=False, use_reloader=False, port=4321) #make sure port is available 

Dash is running on http://127.0.0.1:4321/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4321/ (Press CTRL+C to quit)
